In [86]:
import pandas as pd
import numpy as np
import requests
import json
import scraping_functions
import locale
from graph_tool.all import *
from datetime import datetime
from bs4 import BeautifulSoup
from collections import Counter
locale.setlocale(locale.LC_TIME, 'en_US.UTF-8');

In [87]:
def filt_mus(x):
    try:
        return 'mus' in x.lower()
    except AttributeError:
        return False


In [88]:
with open('data/announcements.json') as file:
    anns = json.load(file)

for id in anns.keys():
    anns[id]['id'] = id
anns_list = [value for value in anns.values()]
df = pd.json_normalize(anns_list)
df.set_index('id', inplace=True)
df.drop('title_artists', inplace=True, axis=1)
df.dropna(inplace=True)
df['date'] = df.announcement_date.apply(lambda x: datetime.strptime(x, '%B %d, %Y'))
# nodes = list(df.subtitle.value_counts().index)
nodes = list(df.subtitle[df.subtitle.apply(filt_mus)].value_counts().index)
nodes_dict = {nodes[i]: i for i in range(len(nodes))}

In [90]:
with open('data/artist_announcements_dict.json') as file:
    arti = json.load(file)
    
'''
all_edges = []
for artist in arti:
    ann_list = artist['announcements']
    ann_list.sort(key = lambda x: datetime.strptime(x['announcement_date'], '%B %d, %Y'))
    all_edges.extend([(ann_list[i]['subtitle'], ann_list[i+1]['subtitle']) for i in range(len(ann_list)-1)])
'''

all_edges = []
for artist in arti:
    ann_list = [x for x in artist['announcements'] if filt_mus(x['subtitle'])]
    ann_list.sort(key = lambda x: datetime.strptime(x['announcement_date'], '%B %d, %Y'))
    all_edges.extend([(ann_list[i]['subtitle'], ann_list[i+1]['subtitle']) for i in range(len(ann_list)-1)])



edges_names_weights_d = dict(Counter(all_edges))
edges_names_l = list(edges_names_weights_d.keys())
edges_id_weights_d = {(nodes_dict[key[0]], nodes_dict[key[1]]) : v for key, v in edges_names_weights_d.items() }
edges_id_l = list(edges_id_weights_d.keys())

In [121]:
g = Graph()
g.add_vertex(len(nodes))
g.vp.name = g.new_vertex_property('string')

for i, v in enumerate(g.vertices()):
    g.vp.name[v] = nodes[i]

g.add_edge_list(edges_id_l)
g.ep.count = g.new_edge_property('int')
for e in g.edges():
    g.ep.count[e] = edges_id_weights_d[(int(e.source()), int(e.target()))]

g.vp.indeg = g.new_vertex_property('int')
g.vp.ingeg = g.get_in_degrees(g.get_vertices())

graph_draw(extract_largest_component(g), output_size=(2000,2000),
           edge_pen_width = prop_to_size(g.ep.count, mi=1, ma=20),
           vertex_size = prop_to_size(g.vp.indeg, mi=5, ma=50))


ValueError: value must be of type PropertyMap, not <class 'numpy.ndarray'>

In [92]:
g.vertices

<bound method Graph.vertices of <Graph object, directed, with 5358 vertices and 110372 edges, 1 internal vertex property, 1 internal edge property, at 0x31cc73290>>